# Data Exploration

---

## 1. Introduction

- Briefly describe the purpose of this notebook.
- Outline the main questions guiding your exploratory analysis.

## 2. Data Loading and Overview

```python
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('../data/raw/logistics_data.csv')

# Display basic info and initial rows
df.info()
df.head()

In [ ]:
# Display summary statistics for numerical features
df.describe()

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot distribution of Total Operational Cost
sns.histplot(df['Total Operational Cost (USD)'], bins=30, kde=True)
plt.title("Distribution of Total Operational Cost (USD)")
plt.show()

In [ ]:
# Calculate and plot correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# End